In [1]:
import requests
import pypyodbc
import time
import pandas as pd
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import urllib
import re
import pyodbc
# from sqlalchemy import create_engine
import pyodbc
# from sqlalchemy import create_engine
import html5lib

# Parameters

In [2]:
Last_Round = 'MW_3'
Next_Week = 4
Upper_Week = 5

# select sink database option ["Az Sql", "Bif_Database"]
sink = "Az Sql"

#### Connect to database

### Az Sql

In [4]:
if sink == "Az SQL":
    SERVER_NAME = 'bifking.database.windows.net'
    DATABASE_NAME = 'SoccerAnalysis'
    USERNAME = 'bifking'
    PASSWORD = 'EnglishSoccer@23'

    driver= '{ODBC Driver 17 for SQL Server}'
    conn_str = (
        f"Driver={driver};"
        f"Server=tcp:{SERVER_NAME};"
        f"Database={DATABASE_NAME};"
        f"Uid={USERNAME};"
        f"Pwd={PASSWORD};"
        f"Encrypt=yes;"
        f"TrustServerCertificate=no;"
        f"Connection Timeout=30;"
    )

    # Create a pyodbc connection object
    conn = pyodbc.connect(conn_str)

    # Create a SQLAlchemy engine object
    engine = create_engine("mssql+pyodbc://", creator=lambda: conn)



### Bif On Premise

In [ ]:
if sink == "Bif_Database"
    SERVER_NAME = 'WS1\\WSBIF'
    DATABASE_NAME = 'SoccerAnalysis'
    USERNAME = 'WS1\\User'
    conn= pypyodbc.connect("""
        Driver={{SQL Server}};
        Server={0};
        Database={1};
        Trusted_Connection=yes;""".format(SERVER_NAME, DATABASE_NAME))

    engine = create_engine("mssql+pyodbc://@"+SERVER_NAME+"/"+DATABASE_NAME+"?trusted_connection=yes&driver=SQL+Server")

# All League's Player passing data

In [ ]:
#creating a class that inherit from the dictionary class for mapping of the name variation
class MissingDict(dict):
    __missing__ = lambda self, key: key
    
map_values = {
    "Wolverhampton Wanderers": "Wolves",
    "West Ham United": "West Ham",
    "Tottenham Hotspur": "Tottenham",
    "Nottingham Forest": "Nott'ham Forest",
    "Newcastle United": "Newcastle Utd",
    "Manchester United": "Manchester Utd",
    "Brighton and Hove Albion": "Brighton",
    "Sheffield United": "Sheffield Utd"
}
mapping = MissingDict(**map_values)

# Functions

In [ ]:
# return list of all column in sqltable
def sourcedestcompare(sqltablename:str, pythondf):
    sql_columnlist= """  
    Select COLUMN_NAME from INFORMATION_SCHEMA.COLUMNS
    where TABLE_NAME = '{0}'

    """.format(sqltablename)
    
    df_sqlcolumnlist = pd.read_sql(sql_columnlist, conn)

    # remove the index column coming from database
    sqllist = df_sqlcolumnlist[1:].values.tolist()
    # convert list of list into list of strings
    sqllist = [''.join(ele) for ele in sqllist]
    
    dflist=pythondf.columns.values.tolist()
    
    # identify difference between sql table and pydataframe
    # what sql has but df doesnt contain
    temp = []
    for x in sqllist:
        if x not in dflist:
            temp.append(x)        
    print(temp)
    # what df contains but sql doesnt have
    temp1 = []
    for x in dflist:
        if x not in sqllist:
            temp1.append(x)        
    print(temp1)
#     return sqllist

In [ ]:
# rename duplicate field from fbref.com
def renameduplicatecolumns(dlist, df: pd.DataFrame):
    for ele in dlist:
        cols = []
        count = 1
        for column in df.columns:
            if column == ele:
                cols.append(f'{ele}_{count}')
                count+=1
                continue
            cols.append(column)
        df.columns = cols

# All players passing stats

In [ ]:
standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standing_url)
soup = BeautifulSoup(data.text, 'lxml')
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if 'squads' in l]
team_urls = [f"https://fbref.com{l}" for l in links]
all_PlayerpassingStat = []


for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-"," ")

    
    data = requests.get(team_url)
    PassingStat = pd.read_html(data.text, match = "Passing ")[0]
    PassingStat.columns = PassingStat.columns.droplevel()
    PassingStat["Club"] = mapping[team_name]
    PassingStat.drop(PassingStat.tail(2).index, inplace=True)
    all_PlayerpassingStat.append(PassingStat)
    time.sleep(2)    

PassingStat_df = pd.concat(all_PlayerpassingStat)

In [ ]:
testlist = ['Cmp','Att','Cmp%']
renameduplicatecolumns(testlist, PassingStat_df)

In [ ]:
sourcedestcompare('PlayerPassingStat', PassingStat_df)

In [ ]:
PassingStat_df["Season"] = "2023/2024"

In [ ]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
PassingStat_df["Round"] = Last_Round
PassingStat_df.to_sql('PlayerPassingStat', engine, if_exists='append')

##### Get League's Squad passing data

In [ ]:
data = requests.get("https://fbref.com/en/comps/9/Premier-League-Stats")


squad_passing = pd.read_html(data.text, match = "Squad Passing ")[0]

squad_passing.columns = squad_passing.columns.droplevel()

In [ ]:
testlist = ['Cmp','Att','Cmp%']
renameduplicatecolumns(testlist, squad_passing)

In [ ]:
squad_passing["Season"] = "2023/2024"

In [ ]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
squad_passing["Round"] = Last_Round
squad_passing.to_sql('SquadPassingStat', engine, if_exists='append')

# All players standard stats

In [ ]:
standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"


data = requests.get(standing_url)


soup = BeautifulSoup(data.text, 'lxml')
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if 'squads' in l]
team_urls = [f"https://fbref.com{l}" for l in links]


all_StandardStat = []

for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-"," ")

    
    data = requests.get(team_url)
    import pandas as pd
    StandardStat = pd.read_html(data.text, match = "Standard Stats ")[0]
    StandardStat.columns = StandardStat.columns.droplevel()
    StandardStat["Club"] = mapping[team_name]
    StandardStat.drop(StandardStat.tail(2).index, inplace=True)
    StandardStat.drop(['Matches'], axis=1,inplace=True)
    all_StandardStat.append(StandardStat)
    time.sleep(2)    

StandardStat_df = pd.concat(all_StandardStat)

In [ ]:
testlist = ['Gls','Ast','xG','xAG']
renameduplicatecolumns(testlist, StandardStat_df)

In [ ]:
sourcedestcompare('PlayerStandardStat', StandardStat_df)

In [ ]:
StandardStat_df = StandardStat_df.drop(['PrgC','PrgP','PrgR'],axis=1)

In [ ]:
# StandardStat_df.columns

In [ ]:
StandardStat_df["Season"] = "2023/2024"

In [ ]:

# Send to database (if i want to replace table add if_exists='replace' to the parameter)
StandardStat_df["Round"] = Last_Round
StandardStat_df.to_sql('PlayerStandardStat', engine, if_exists='append')

##### Get League's Squad standart stat

In [ ]:
data = requests.get("https://fbref.com/en/comps/9/Premier-League-Stats")
squad_standard = pd.read_html(data.text, match = "Squad Standard Stats ")[0]
squad_standard.columns = squad_standard.columns.droplevel()
# chanages the duplicate column header at the same time. NO GOOD FOR WHAT I WANT
# squad_standard.rename(columns={squad_standard.columns[15]: "Per90Gls", squad_standard.columns[16]: "Per90Ast"}, inplace=True)
# squad_standard

In [ ]:
testlist = ['Gls','Ast','xG','xAG']
renameduplicatecolumns(testlist, squad_standard)

In [ ]:
sourcedestcompare('SquadStandardStat', squad_standard)

In [ ]:
squad_standard = squad_standard.drop(['PrgC','PrgP'],axis=1)

In [ ]:
squad_standard["Season"] = "2023/2024"

In [ ]:

# Send to database (if i want to replace table add if_exists='replace' to the parameter)
squad_standard["Round"] = Last_Round
squad_standard.to_sql('SquadStandardStat', engine, if_exists='append')

# All League's Player shooting data

In [ ]:
standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standing_url)
soup = BeautifulSoup(data.text, 'lxml')
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if 'squads' in l]
team_urls = [f"https://fbref.com{l}" for l in links]
all_PlayershootingStat = []


for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-"," ")

    
    data = requests.get(team_url)
    ShootingStat = pd.read_html(data.text, match = "Shooting ")[0]
    ShootingStat.columns = ShootingStat.columns.droplevel()
    ShootingStat["Club"] = mapping[team_name]
    ShootingStat.drop(ShootingStat.tail(2).index, inplace=True)

    all_PlayershootingStat.append(ShootingStat)
    time.sleep(2)    

ShootingStat_df = pd.concat(all_PlayershootingStat)  

In [ ]:
ShootingStat_df["Season"] = "2023/2024"

In [ ]:

# Send to database (if i want to replace table add if_exists='replace' to the parameter)
ShootingStat_df["Round"] = Last_Round
ShootingStat_df.to_sql('PlayerShootingStat', engine, if_exists='append')

##### Get League's Squad shooting data

In [ ]:
data = requests.get("https://fbref.com/en/comps/9/Premier-League-Stats")
squad_shooting = pd.read_html(data.text, match = "Shooting ")[0]
squad_shooting.columns = squad_shooting.columns.droplevel()
# squad_shooting

In [ ]:
squad_shooting["Season"] = "2023/2024"

In [ ]:

# Send to database (if i want to replace table add if_exists='replace' to the parameter)
squad_shooting["Round"] = Last_Round
squad_shooting.to_sql('SquadShootingStat', engine, if_exists='append')

# All League's Player passtype data

In [ ]:
standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standing_url)
soup = BeautifulSoup(data.text, 'lxml')
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if 'squads' in l]
team_urls = [f"https://fbref.com{l}" for l in links]


all_PlayerpasstypeStat = []


for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-"," ")
    data = requests.get(team_url)
    
    PasstypeStat = pd.read_html(data.text, match = "Pass Types ")[0]
    PasstypeStat.columns = PasstypeStat.columns.droplevel()
    PasstypeStat["Club"] = mapping[team_name]
    PasstypeStat.drop(PasstypeStat.tail(2).index, inplace=True)

    all_PlayerpasstypeStat.append(PasstypeStat)
    time.sleep(2)    

PasstypeStat_df = pd.concat(all_PlayerpasstypeStat)  

In [ ]:
PasstypeStat_df["Season"] = "2023/2024"

In [ ]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
PasstypeStat_df["Round"] = Last_Round
PasstypeStat_df.to_sql('PlayerPasstypeStat', engine, if_exists='append')

### Get League's Squad passtype data

In [ ]:
data = requests.get("https://fbref.com/en/comps/9/Premier-League-Stats")
squad_passtype = pd.read_html(data.text, match = "Pass Types ")[0]
squad_passtype.columns = squad_passtype.columns.droplevel()
# squad_passtype

In [ ]:
squad_passtype["Season"] = "2023/2024"

In [ ]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
squad_passtype["Round"] = Last_Round
squad_passtype.to_sql('SquadPasstypeStat', engine, if_exists='append')

# All League's Player chances data

In [ ]:
standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standing_url)
soup = BeautifulSoup(data.text, 'lxml')
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if 'squads' in l]
team_urls = [f"https://fbref.com{l}" for l in links]


all_PlayerchancesStat = []

for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-"," ")
    data = requests.get(team_url)
    
    ChancesStat = pd.read_html(data.text, match = "Goal and Shot Creation ")[0]
    ChancesStat.columns = ChancesStat.columns.droplevel()
    ChancesStat["Club"] = mapping[team_name]
    ChancesStat.drop(ChancesStat.tail(2).index, inplace=True)

    all_PlayerchancesStat.append(ChancesStat)
    time.sleep(2)    

ChancesStat_df = pd.concat(all_PlayerchancesStat)  

In [ ]:
testlist = ['PassLive','PassDead','TO','Sh','Fld','Def']
renameduplicatecolumns(testlist, ChancesStat_df)

In [ ]:
sourcedestcompare('PlayerChancesStat', ChancesStat_df)

In [ ]:
ChancesStat_df["Season"] = "2023/2024"

In [ ]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
ChancesStat_df["Round"] = Last_Round
ChancesStat_df.to_sql('PlayerChancesStat', engine, if_exists='append')

##### Get League's Squad chances data

In [ ]:
data = requests.get("https://fbref.com/en/comps/9/Premier-League-Stats")
squad_chances = pd.read_html(data.text, match = "Goal and Shot Creation ")[0]
squad_chances.columns = squad_chances.columns.droplevel()
# squad_chances

In [ ]:
testlist = ['PassLive','PassDead','TO','Sh','Fld','Def']
renameduplicatecolumns(testlist, squad_chances)

In [ ]:
squad_chances["Season"] = "2023/2024"

In [ ]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
squad_chances["Round"] = Last_Round
squad_chances.to_sql('SquadChancesStat', engine, if_exists='append')

# All League's Player defensiveaction data

In [ ]:
standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standing_url)
soup = BeautifulSoup(data.text, 'lxml')
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if 'squads' in l]
team_urls = [f"https://fbref.com{l}" for l in links]


all_PlayerdefensiveactionStat = []

for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-"," ")
    data = requests.get(team_url)
    
    DefensiveactionStat = pd.read_html(data.text, match = "Defensive Actions ")[0]
    DefensiveactionStat.columns = DefensiveactionStat.columns.droplevel()
    DefensiveactionStat["Club"] = mapping[team_name]
    DefensiveactionStat.drop(DefensiveactionStat.tail(2).index, inplace=True)

    all_PlayerdefensiveactionStat.append(DefensiveactionStat)
    time.sleep(2)    

DefensiveactionStat_df = pd.concat(all_PlayerdefensiveactionStat)  

In [ ]:
testlist = ['Tkl']
renameduplicatecolumns(testlist, DefensiveactionStat_df)

In [ ]:
sourcedestcompare('PlayerDefensiveactionStat', DefensiveactionStat_df)

In [ ]:
DefensiveactionStat_df.columns.values.tolist()

In [ ]:
DefensiveactionStat_df["Season"] = "2023/2024"

In [ ]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
DefensiveactionStat_df["Round"] = Last_Round
DefensiveactionStat_df.to_sql('PlayerDefensiveactionStat', engine, if_exists='append')

##### Get League's Squad defensiveaction data

In [ ]:
data = requests.get("https://fbref.com/en/comps/9/Premier-League-Stats")
squad_defensiveaction = pd.read_html(data.text, match = "Defensive Actions ")[0]
squad_defensiveaction.columns = squad_defensiveaction.columns.droplevel()
# squad_defensiveaction

In [ ]:
testlist = ['Tkl']
renameduplicatecolumns(testlist, squad_defensiveaction)

In [ ]:
sourcedestcompare('SquadDefensiveactionStat', squad_defensiveaction)

In [ ]:
squad_defensiveaction["Season"] = "2023/2024"

In [ ]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
squad_defensiveaction["Round"] = Last_Round
squad_defensiveaction.to_sql('SquadDefensiveactionStat', engine, if_exists='append')

# All League's Player possession data

In [ ]:
standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standing_url)
soup = BeautifulSoup(data.text, 'lxml')
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if 'squads' in l]
team_urls = [f"https://fbref.com{l}" for l in links]


all_PlayerpossessionStat = []

for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-"," ")
    data = requests.get(team_url)
    
    PossessionStat = pd.read_html(data.text, match = "Possession ")[0]
    PossessionStat.columns = PossessionStat.columns.droplevel()
    PossessionStat["Club"] = mapping[team_name]
    PossessionStat.drop(PossessionStat.tail(2).index, inplace=True)

    all_PlayerpossessionStat.append(PossessionStat)
    time.sleep(2)    

PossessionStat_df = pd.concat(all_PlayerpossessionStat)  

In [ ]:
sourcedestcompare('PlayerPossessionStat', PossessionStat_df)

In [ ]:
PossessionStat_df["Season"] = "2023/2024"

In [ ]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
PossessionStat_df["Round"] = Last_Round
PossessionStat_df.to_sql('PlayerPossessionStat', engine, if_exists='append')

##### Get League's Squad possession data

In [ ]:
data = requests.get("https://fbref.com/en/comps/9/Premier-League-Stats")
squad_possession = pd.read_html(data.text, match = "Possession ")[0]
squad_possession.columns = squad_possession.columns.droplevel()
# squad_possession

In [ ]:
sourcedestcompare('SquadPossessionStat', squad_possession)

In [ ]:
squad_possession["Season"] = "2023/2024"

In [ ]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
squad_possession["Round"] = Last_Round
squad_possession.to_sql('SquadPossessionStat', engine, if_exists='append')

# All League's Player playingtime data

In [ ]:
standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standing_url)
soup = BeautifulSoup(data.text, 'lxml')
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if 'squads' in l]
team_urls = [f"https://fbref.com{l}" for l in links]


all_PlayerplayingtimeStat = []

In [ ]:
for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-"," ")
    data = requests.get(team_url)
    
    PlayingtimeStat = pd.read_html(data.text, match = "Playing Time ")[0]
    PlayingtimeStat.columns = PlayingtimeStat.columns.droplevel()
    PlayingtimeStat["Club"] = mapping[team_name]
    PlayingtimeStat.drop(PlayingtimeStat.tail(2).index, inplace=True)

    all_PlayerplayingtimeStat.append(PlayingtimeStat)
    time.sleep(2)    

PlayingtimeStat_df = pd.concat(all_PlayerplayingtimeStat)  

In [ ]:
sourcedestcompare('PlayerPlayingtimeStat', PlayingtimeStat_df)

In [ ]:
PlayingtimeStat_df["Season"] = "2023/2024"

In [ ]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
PlayingtimeStat_df["Round"] = Last_Round
PlayingtimeStat_df.to_sql('PlayerPlayingtimeStat', engine, if_exists='append')

##### Get League's Squad playingtime data

In [ ]:
data = requests.get("https://fbref.com/en/comps/9/Premier-League-Stats")
squad_playingtime = pd.read_html(data.text, match = "Playing Time ")[0]
squad_playingtime.columns = squad_playingtime.columns.droplevel()
# squad_playingtime

In [ ]:
squad_playingtime["Season"] = "2023/2024"

In [ ]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
squad_playingtime["Round"] = Last_Round
squad_playingtime.to_sql('SquadPlayingtimeStat', engine, if_exists='append')

# All League's Player miscellaneous  data

In [ ]:
standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standing_url)
soup = BeautifulSoup(data.text, 'lxml')
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if 'squads' in l]
team_urls = [f"https://fbref.com{l}" for l in links]


all_PlayermiscellaneousStat = []

for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-"," ")
    data = requests.get(team_url)
    
    MiscellaneousStat = pd.read_html(data.text, match = "Miscellaneous Stats ")[0]
    MiscellaneousStat.columns = MiscellaneousStat.columns.droplevel()
    MiscellaneousStat["Club"] = mapping[team_name]
    MiscellaneousStat.drop(MiscellaneousStat.tail(2).index, inplace=True)

    all_PlayermiscellaneousStat.append(MiscellaneousStat)
    time.sleep(2)    

MiscellaneousStat_df = pd.concat(all_PlayermiscellaneousStat)  

In [ ]:
MiscellaneousStat_df["Season"] = "2023/2024"

In [ ]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
MiscellaneousStat_df["Round"] = Last_Round
MiscellaneousStat_df.to_sql('PlayerMiscellaneousStat', engine, if_exists='append')

##### Get League's Squad miscellaneous data

In [ ]:
data = requests.get("https://fbref.com/en/comps/9/Premier-League-Stats")
squad_miscellaneous = pd.read_html(data.text, match = "Miscellaneous Stats ")[0]
squad_miscellaneous.columns = squad_miscellaneous.columns.droplevel()
# squad_miscellaneous

In [ ]:
squad_miscellaneous["Season"] = "2023/2024"

In [ ]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
squad_miscellaneous["Round"] = Last_Round
squad_miscellaneous.to_sql('SquadMiscellaneousStat', engine, if_exists='append')

###### List of all club players name(include some amount that has been transfered out of the club)

In [ ]:
# Playernames = StandardStat_df[["Player","Club"]].reset_index(drop=True)
# Playernames

In [ ]:
#Send to database 
# Playernames.to_sql('SquadPlayerName', engine,if_exists='replace' )

# Total goals and assist contribution for the last 5 matches

In [ ]:
standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standing_url)
soup = BeautifulSoup(data.text, 'lxml')
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if 'squads' in l]
team_urls = [f"https://fbref.com{l}" for l in links]

# CHANGE n>-4 to n>-5

In [ ]:

# team_urls = [l for l in team_urls if l not in ['https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats','https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats','https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats','https://fbref.com/en/squads/e4a775cb/Nottingham-Forest-Stats','https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats','https://fbref.com/en/squads/19538871/Manchester-United-Stats','https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats']]


all_PlayerLast5Performance = []


for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-"," ")
# team_url = team_urls[0]
# team_url
    data = requests.get(team_url)
    soup = BeautifulSoup(data.text, 'lxml')
    scorefixture_table = soup.select('table.stats_table')[1]
    links = scorefixture_table.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if '/matches' in l ]
    # remove duplicate from list
    links = list(dict.fromkeys(links))
    # removing non premier league (Remove europa)
    links = [l for l in links if "Premier" in l]
    links
    
    
    last5_marchreport = []
    n = -1    
    while n >-6:
        # check, especially within the first 5matches of the season, that n should be within the index range
        if n >= -len(links):  # Check if n is within the valid index range
            data = requests.get(f"https://fbref.com{links[n]}")
            summary = pd.read_html(data.text, match= mapping[team_name]+" Player Stats Table")[0]
            summary.columns = summary.columns.droplevel()
            # retrun the opposition team name
            string = links[n]
            modifiedstring = string[string.rindex('/') + 1:]
            words = re.findall(r'\b\w+\b', modifiedstring)
            # Remove the last 5 words
            summary["Clash"] = ' '.join(words[:-5])



            summary["Club_"] = team_name
            #create a short name
            summary["ShClub_"] = mapping[team_name]
            summary.drop(summary.tail(1).index, inplace=True)
        # if it is out of range then move to the next loop
        else:
            n -= 1
            continue
        
        n -= 1
        
        last5_marchreport.append(summary)
        time.sleep(2)

    pd_summary = pd.concat(last5_marchreport)
    all_PlayerLast5Performance.append(pd_summary)
    time.sleep(2)

pd_allteam = pd.concat(all_PlayerLast5Performance)

In [ ]:
# pd_allteam

In [ ]:
# Send the granula data for analysis the games where a player socred
dfperformanceagaisntclub = pd_allteam[['Player','#','Min','Gls','Ast','Sh','SoT','CrdY','CrdR','Blocks', 'Tkl', 'Int','SCA','GCA','Club_','ShClub_', 'Clash']]
dfperformanceagaisntclub["Season"] = "2023/2024"
dfperformanceagaisntclub["Round"] = Last_Round
dfperformanceagaisntclub.to_sql('_PlayerPerformanceRecentGran', engine, if_exists='append')

In [ ]:
dfperformancerecent = pd_allteam[['Player','#','Min','Gls','Ast','Sh','SoT','CrdY','CrdR','Blocks', 'Tkl', 'Int','SCA','GCA','Club_','ShClub_']]
dfperformancerecent = dfperformancerecent.groupby(['#','Player','Club_', 'ShClub_']).sum().reset_index()
# dff = dfperformancerecent.groupby(['Club_'])
# dff.get_group("West Ham United")


In [ ]:
dfperformancerecent["Season"] = "2023/2024"

In [ ]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
dfperformancerecent["Round"] = Last_Round
dfperformancerecent.to_sql('_PlayerPerformanceRecent', engine, if_exists='append')

# Team Result

In [ ]:
standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standing_url)
soup = BeautifulSoup(data.text, 'lxml')
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if 'squads' in l]
team_urls = [f"https://fbref.com{l}" for l in links]


all_TeamresultStat = []

In [ ]:
for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-"," ")
    data = requests.get(team_url)
    
    ResultStat = pd.read_html(data.text, match = "Scores & Fixtures ")[0]
#     ResultStat.columns = ResultStat.columns.droplevel()
    ResultStat["Club"] = team_name
    ResultStat["ShClub_"] = mapping[team_name]
#     ResultStat.drop(ResultStat.tail(2).index, inplace=True)

    all_TeamresultStat.append(ResultStat)
    time.sleep(2)    
    

ResultStat_df = pd.concat(all_TeamresultStat)  

In [ ]:
ResultStat_df = ResultStat_df[ResultStat_df['Result'].notna()]
ResultStat_df = ResultStat_df[ResultStat_df['Comp']=='Premier League']

In [ ]:



#Dont run
# Removing rows where a column carries a particular string
# dd = ResultStat_df[ResultStat_df['Result'].str.contains('Result') == False]
# dd

In [ ]:
ResultStat_df["Season"] = "2023/2024"

In [ ]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
ResultStat_df =ResultStat_df.rename({"Round":"MatchWeek"}, axis='columns')
ResultStat_df["Round"] = Last_Round
ResultStat_df.to_sql('FixturesStat', engine, if_exists='append')

# Table Standings

In [ ]:
standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standing_url)
Standingtable = pd.read_html(data.text, match = "Regular season Table")[0]
# Standingtable[Standingtable['Squad']]

In [ ]:
Standingtable_df = Standingtable[['Rk','Squad','W','D','L','Pts','GF','GA','GD','Last 5','Top Team Scorer','MP']]

In [ ]:
Standingtable_df["Season"] = "2023/2024"

In [ ]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
Standingtable_df["Round"] = Last_Round
Standingtable_df.to_sql('_Standtable', engine, if_exists='append')

# Next 7 fixtures

In [ ]:
standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standing_url)
soup = BeautifulSoup(data.text, 'lxml')
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if 'squads' in l]
team_urls = [f"https://fbref.com{l}" for l in links]
# team_urls = [l for l in team_urls if l not in ['https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats','https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats','https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats','https://fbref.com/en/squads/e4a775cb/Nottingham-Forest-Stats','https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats','https://fbref.com/en/squads/19538871/Manchester-United-Stats','https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats']]


all_Next7Fixture = []


for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-"," ")
#     team_url = team_urls[0]
    # team_url
    data = requests.get(team_url)
    fixtures = pd.read_html(data.text, match= "Scores & Fixtures ")[0]
    fixtures["shClub"] = mapping[team_name]
    fixtures = fixtures[fixtures['Comp']=='Premier League']
    fixtures[['Match Report']] = fixtures[['Match Report']].fillna('')
    fixtures = fixtures[fixtures['Match Report'].str.contains('Match Report')==False]
    
    fixtures = fixtures.iloc[:7]
# converting object type to date data type
    #     fixtures["Date"] = pd.to_datetime(fixtures["Date"])
#     fixtures.nsmallest(n=7, columns=['Date'])
    
    all_Next7Fixture.append(fixtures)
    time.sleep(2)
    
pd_allNext7Fixtures = pd.concat(all_Next7Fixture)
# pd_allNext7Fixtures  
    


In [ ]:
pd_allNext7Fixtures["Season"] = "2023/2024"

In [ ]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
pd_allNext7Fixtures["Round"] = Last_Round
pd_allNext7Fixtures.to_sql('Next7Fixtures', engine, if_exists='append')

# This week and Next week Fixture draw

#### Edit filter condition within the first block of code

<font color='blue'>Edit filter condition within the first block of code</font>

In [ ]:
nextweekfixture_url = "https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures"
data = requests.get(nextweekfixture_url)
nextfixtures = pd.read_html(data.text, match= "Scores & Fixtures ")[0]
nextfixtures = nextfixtures[(nextfixtures["Match Report"]=="Head-to-Head")&((nextfixtures["Wk"]==Next_Week)|(nextfixtures["Wk"]==Upper_Week))]
# #requires that the column changes field type : I was changing from float to string. 
# Week = {1.0: 'This week', 2.0: 'Next week'}
# nextfixtures = nextfixtures.replace({"Wk":Week})
# #creating a new column entirely
nextfixtures['Week'] = ['This week' if x==Next_Week else 'Next week' for x in nextfixtures['Wk']]
nextfixtures

In [ ]:
nextfixtures["Season"] = "2023/2024"

In [ ]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
nextfixtures["Round"] = Last_Round
nextfixtures.to_sql('NextweekFixtures', engine, if_exists='append')

# Get Player HeadShot

In [ ]:
# code
# standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
# data = requests.get(standing_url)
# soup = BeautifulSoup(data.text, 'lxml')
# standings_table = soup.select('table.stats_table')[0]
# links = standings_table.find_all('a')
# links = [l.get("href") for l in links]
# links = [l for l in links if 'squads' in l]
# team_urls = [f"https://fbref.com{l}" for l in links][0]
# all_PlayershootingStat = []



In [ ]:
# for team_url in team_urls:

#     team_url = team_urls[0]
# team_url

# code
# data = requests.get(team_urls)
# soup = BeautifulSoup(data.text, 'lxml')
# standings_table = soup.select('table.stats_table')[0]
# links = standings_table.find_all('a')
# links = [l.get("href") for l in links]
# links = [l for l in links if 'players' in l if 'matchlogs' not in l]
# player_urls = [f"https://fbref.com{l}" for l in links][0]
# player_urls
# Player_name = player_urls.split("/")[-1].replace("-"," ")
# all_playerphoto = []

# # for player_url in player_url:
# data =urllib.request.urlopen(player_urls)
# soup= BeautifulSoup(data)
# images = soup.findAll('img')
# images = [i for i in images if 'headshots' in i['src']]
    
# allplayer = []

In [ ]:
# code
# for image in images:
#     a=image['src']
    
#     allplayer.append(a)
# df = pd.DataFrame(allplayer, columns = ['imageurl'])
# df['name'] = Player_name
# df

In [ ]:
# code
# standing_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
# data = requests.get(standing_url)
# soup = BeautifulSoup(data.text, 'lxml')
# standings_table = soup.select('table.stats_table')[0]
# links = standings_table.find_all('a')
# links = [l.get("href") for l in links]
# links = [l for l in links if 'squads' in l]
# team_urls = [f"https://fbref.com{l}" for l in links]

# all_teamplayerphoto = []
# for team_url in team_urls:
#     team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-"," ")
#     data = requests.get(team_url)
#     soup = BeautifulSoup(data.text, 'lxml')
#     standings_table = soup.select('table.stats_table')[0]
#     links = standings_table.find_all('a')
#     links = [l.get("href") for l in links]
#     links = [l for l in links if 'players' in l if 'matchlogs' not in l]
#     player_urls = [f"https://fbref.com{l}" for l in links]
    
    
#     all_playerphoto = []
#     for player_url in player_urls:
#         Player_name = player_url.split("/")[-1].replace("-"," ")
#         data =urllib.request.urlopen(player_url)
#         soup= BeautifulSoup(data)
#         images = soup.findAll('img')
#         images = [i for i in images if 'headshots' in i['src']]
#         player = []
#         for image in images:
#             a=image['src']
#             player.append(a)
#             df = pd.DataFrame(player, columns = ['imageurl'])
#             df['name'] = Player_name
# #             time.sleep(0.5)
#         pd_player =df
#         pd_player['Squad'] = mapping[team_name]
        
        
#         all_playerphoto.append(pd_player)
# #         time.sleep(0.5)
#     pd_all_playerphoto = pd.concat(all_playerphoto)
    
    
#     all_teamplayerphoto.append(pd_all_playerphoto)
#     time.sleep(0.5)
# pd_all_teamplayerphoto = pd.concat(all_teamplayerphoto)
        

In [ ]:
# code
# pd_all_teamplayerphoto

In [ ]:
# Send to database (if i want to replace table add if_exists='replace' to the parameter)
# nextfixtures["Round"] = 'MW_10'


# code
# pd_all_teamplayerphoto.to_sql('_Playerphoto', eng, if_exists='')